In [299]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [300]:
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1200
CALORIES_PER_DAY = 2000
DAYS_PER_MENU = 7
DISHES_PER_DAY = 3

PENALTY = 400
POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 3000

REAL_DATA = True

#### Helper functions

In [301]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Test data

In [302]:
def fake_data():
    dishes = {}
    for _ in range(DISH_COUNT):
        name = ''.join(random.choices(string.ascii_lowercase, k=6))
        dishes[name] = random.randint(MIN_CALORIES_PER_DISH, MAX_CALORIES_PER_DISH)
    dishes = list(dishes.items())
    return dishes

#### Real data

In [303]:
def real_data():
    dishes = {}
    data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
    df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])

    while len(dishes) != DISH_COUNT:
        random_choice = df.sample()
        if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
            dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
    dishes = list(dishes.items())
    return dishes

In [304]:
dishes = {}
if REAL_DATA:
    dishes = real_data()
else:
    dishes = fake_data()
print(*dishes[:7], sep='\n')

('Apple Candied', 255.42)
('Menudo Soup Home Recipe', 118.09)
('Kfc Fried Chicken Extra Crispy Skin And Breading', 464.0)
('Tortellini Cheese-Filled No Sauce', 354.0)
('Peanuts Virginia Oil-Roasted Without Salt', 826.54)
('Beef Noodles And Vegetables Including Carrots Broccoli And/or Dark-Green Leafy; Tomato-Based Sauce', 311.25)
('Lobster Bisque', 128.96)


#### Sample random menu

In [305]:
def create_random_menu():
    menu = np.ndarray((7, 3), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [306]:
def count_calories_per_day(menu):
    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]

    return calories


def count_attr_per_day(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    names_dict = dict(zip(unique, counts))

    attr = []
    for day in range(DAYS_PER_MENU):
        attr.append(0)
        for dish_index in range(DISHES_PER_DAY):
            attr[-1] += (names_dict[menu[day, dish_index]] - 1) * PENALTY

    return attr


def score_menu(menu):
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    penalty = [attr ** 2 for attr in count_attr_per_day(menu)]
    squares = [calories ** 2 for calories in differences]

    return round(sum(squares + penalty))


def check_duplications(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    is_repeating = dict(zip(unique, counts))

    for elem in is_repeating:
        if is_repeating[elem] > 2:
            print("duplicants detected")
            return
    print("No duplicants")

In [307]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 13869177
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                     │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    659.51  │ [('Ham Or Pork Noodles And Vegetables Including Carrots Broccoli And/or Dark-Green Leafy; Tomato-Based Sauce', 311.25)     │
│       │            │  ('Stewed Pink Beans With White Potatoes And Ham Puerto Rican Style', 219.3)                                               │
│       │            │  ('Lobster Bisque', 128.96)]                                                                                               │
├───────┼────────────┼──────────────────────────────────────────────────────────────────────────

### Algorithm

In [308]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [309]:
def initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        population.append(create_random_menu())
    return population

In [310]:
def genetic_algorithm():
    population = initial_population()
    for i in range(ITERATIONS):
        population.sort(key=score_menu)
        mating_population = population[:MATING_POPULATION_SIZE]

        offspring_population = []
        for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
            offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
            if random.random() < 0.1:
                offspring_population[-1] = random_mutate(offspring_population[-1])
            if random.random() < 0.1:
                offspring_population[-1] = swap_mutate(offspring_population[-1])

        population = mating_population + offspring_population

    population.sort(key=score_menu)
    best_individual, worst_individual = population[0], population[-1]

    return best_individual, worst_individual

### Example

In [311]:
best_individual, worst_individual = genetic_algorithm()
visualize_menu(best_individual)
check_duplications(best_individual)

Score: 21545
╒═══════╤════════════╤═══════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                    │
╞═══════╪════════════╪═══════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    1997.08 │ [('Semisweet Chocolate Made With Butter', 810.9)                                          │
│       │            │  ('Chilaquiles Tortilla Casserole With Salsa And Cheese No Egg', 693.68)                  │
│       │            │  ('Pasta Whole Grain With Cream Sauce Restaurant', 492.5)]                                │
├───────┼────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │    1942.78 │ [('On The Border Soft Taco With Ground Beef Cheese And Lettuce', 741.96)                  │
│       │            │  ('Wrap Sandwich Filled With Meat Poultry Or

### Experiments

In [312]:
def visualize_experiment(parameters, measurements):
    headers = [parameters, "Best", "Worst"]
    table = []
    for measurement in measurements:
        table.append(measurement)
    print(tabulate(table, headers, tablefmt="fancy_grid"))

In [313]:
def experiment_population_size():
    results = []
    for population_size in [10, 20, 50, 100, 150, 250]:
        POPULATION_SIZE = population_size
        MATING_POPULATION_SIZE = population_size // 2
        best_individual, worst_individual = genetic_algorithm()
        results.append([population_size, score_menu(best_individual), score_menu(worst_individual)])
    return results


visualize_experiment("Population size", experiment_population_size())

╒═══════════════════╤════════╤═════════╕
│   Population size │   Best │   Worst │
╞═══════════════════╪════════╪═════════╡
│                10 │    787 │  605276 │
├───────────────────┼────────┼─────────┤
│                20 │    965 │  247339 │
├───────────────────┼────────┼─────────┤
│                50 │   1298 │  290709 │
├───────────────────┼────────┼─────────┤
│               100 │   1736 │  124836 │
├───────────────────┼────────┼─────────┤
│               150 │    853 │  320937 │
├───────────────────┼────────┼─────────┤
│               250 │   1233 │    1233 │
╘═══════════════════╧════════╧═════════╛


In [314]:
def experiment_iterations():
    results = []
    for iterations in [10, 20, 50, 100, 150, 250, 500, 1000, 2000]:
        ITERATIONS = iterations
        best_individual, worst_individual = genetic_algorithm()
        results.append([iterations, score_menu(best_individual), score_menu(worst_individual)])
    return results


visualize_experiment("Iterations", experiment_iterations())

╒══════════════╤════════╤═════════╕
│   Iterations │   Best │   Worst │
╞══════════════╪════════╪═════════╡
│           10 │   9753 │   38962 │
├──────────────┼────────┼─────────┤
│           20 │   1725 │  107627 │
├──────────────┼────────┼─────────┤
│           50 │  11793 │  592188 │
├──────────────┼────────┼─────────┤
│          100 │   2225 │  679935 │
├──────────────┼────────┼─────────┤
│          150 │   6298 │  241119 │
├──────────────┼────────┼─────────┤
│          250 │   1794 │   36018 │
├──────────────┼────────┼─────────┤
│          500 │   1271 │  213455 │
├──────────────┼────────┼─────────┤
│         1000 │   8050 │  109461 │
├──────────────┼────────┼─────────┤
│         2000 │   9250 │  696373 │
╘══════════════╧════════╧═════════╛
